# Commodity Dashboard

## Importing Data, Libraries and Scripts

Data is stored into commodity_lab/db/commodities.db 

In [8]:
import sys
print(sys.executable)

c:\Users\dgalassi\commodity_lab\env_commodity_lab\Scripts\python.exe


In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as ply
import plotly.express as px
import seaborn as sns
import sys

In [ ]:
#update the database with the most recent data
sys.path.append('../../scripts')
from update_commodities_data import update_commodities

#tickers i want to update
tickers = {
    "Gold": "GC=F",
    "Silver": "SI=F",
    "Crude Oil": "CL=F",
    "Natural Gas": "NG=F",
    "Copper": "HG=F",
    "Corn": "ZC=F",
    "Wheat": "ZW=F"
}
update_commodities(tickers)

In [15]:
#connect to commodities db 
import sqlite3 as sql

#enstablish a connection
conn = sql.connect("../../db/commodities.db")
df = pd.read_sql('SELECT * FROM prices',conn)
conn.close()

## Analyze the Dataset

1. Price Trends & Returns

Line charts of price history for key commodities

% change over multiple time horizons (1 day, 5 days, 1 month, 1 year)

Quick summary stats (last price, daily return, volatility)

2. Volatility & Risk Metrics

Rolling volatility (e.g., 30-day std dev)

Maybe Value at Risk (VaR) or drawdowns for the bigger picture risk view

3. Correlations

Heatmap or matrix showing correlation between commodities over the past month or quarter — helps PMs see diversification or contagion

4. Volume & Liquidity

Volume trends, so they know where the market interest is

5. Alerts or Significant Moves

Highlight big moves or breakouts (e.g., +3% daily move)

Could be a simple “flag” or conditional formatting

6. Comparative Analysis

Commodities side-by-side to see which are outperforming

### Price Trends and Returns

In [ ]:
df_prices = df[['date','name','close']]
df_prices['date'] = pd.to_datetime(df.date)

In [ ]:
#pivot to wide so we have each commodity in a column
df_wide = df.pivot(index='date', columns='name', values='close')
#calculate returns, reconvert to wide and drop NaN
df_daily = df_wide.pct_change(1).reset_index().melt(id_vars='date', var_name='name', value_name='return').dropna()
df_weekly = df_wide.pct_change(5).reset_index().melt(id_vars='date', var_name='name', value_name='return').dropna()
df_monthly = df_wide.pct_change(21).reset_index().melt(id_vars='date', var_name='name', value_name='return').dropna()

In [61]:
#line plots

fig_close_prices = px.line(df_prices, x='date', y='close', color='name',title='Closing Prices')
fig_close_prices.show()

fig_daily_return = px.line(df_daily, x='date', y='return', color='name',title='Daily Returns')
fig_daily_return.show()

fig_weekly_return = px.line(df_weekly, x='date', y='return', color='name',title='Weekly Returns')
fig_weekly_return.show()

fig_monthly_return = px.line(df_monthly, x='date', y='return', color='name',title='Monthly Returns')
fig_monthly_return.show()

In [68]:
#box plots

fig_daily_box_return = px.box(df_daily,x='name',y='return',points='all',
                              title='Daily Returns Distribution by Commodity')
fig_daily_box_return.show()


fig_weekly_box_return = px.box(df_weekly,x='name',y='return',points='all',
                              title='Weekly Returns Distribution by Commodity')
fig_weekly_box_return.show()


fig_monthly_box_return = px.box(df_monthly,x='name',y='return',points='all',
                              title='Monthly Returns Distribution by Commodity')
fig_monthly_box_return.show()